In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

import os

llm = ChatOpenAI(base_url='https://api.deepseek.com', api_key=os.getenv('DEEPSEEK_SK'), model='deepseek-chat')

# 将提示模板 和 模型 链接在一起
prompt = ChatPromptTemplate.from_template('跟我说一个关于{topic}的笑话')

chain = prompt | llm | StrOutputParser()
# result = chain.invoke({'topic': '冰激凌'})
# print(f'结果: {result}')

prompt_value = prompt.invoke({'topic': '冰激凌'})
messages = llm.invoke(prompt_value)
output = StrOutputParser().invoke(messages)

print(prompt_value)
print('\n\n')
print(messages)
print('\n\n')
print(output)
print('\n\n')



messages=[HumanMessage(content='跟我说一个关于冰激凌的笑话', additional_kwargs={}, response_metadata={})]



content='当然！这里有一个冰激凌笑话：\n\n---\n\n一个草莓冰淇淋和一個巧克力冰淇淋在街上走路。\n\n突然，巧克力冰淇淋不小心摔了一跤，滚到了路边。\n\n它一边爬起来一边抱怨，可旁边的草莓冰淇淋却一动不动，站在那里哈哈大笑。\n\n巧克力冰淇淋很生气地问：“我都摔成这样了，你怎么还笑？”\n\n草莓冰淇淋得意地说：\n\n“因为……我我是‘草莓’（挺没）事啊！”\n\n---\n\n（谐音梗冷笑话，希望让你嘴角上扬一下~ 😄）' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 12, 'total_tokens': 114, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 12}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '148937a7-0d06-44e9-b163-d6be00313b16', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--770e0438-b131-4409-bf85-cfe7e69ddd8c-0' usage_metadata={'input_tokens': 12, 'output_tokens': 102, 'total_tokens': 114, 'input_token_details': {'cache_read': 0},

In [11]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough


# RunnablePassthrough()单独调用时：简单接收输入并传递
# RunnablePassthrough.assign(...)，将接收输入，并添加传递给assign函数的额外参数

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x['num']*3),
    modified=lambda x: x['num']+1,
)

result = runnable.invoke({'num': 1})
print(result)


{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}


In [17]:
prompt = ChatPromptTemplate.from_template('请对{question}这个问题进行简单的回答')

retrieval_chain = (
    {'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

result = retrieval_chain.invoke('在做什么事')
print(f'问题: {prompt}\n\n回答: {result}')

问题: input_variables=['question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='请对{question}这个问题进行简单的回答'), additional_kwargs={})]

回答: 好的，这是一个非常灵活的问题，可以根据不同的情境进行简单回答。

**1. 最直接的日常回答：**
*   “没做什么。”（当比较空闲时）
*   “在看手机/看书。”
*   “在休息。”
*   “刚吃完饭。”
*   “在等人。”

**2. 正在处理具体事务的回答：**
*   “在工作/上班。”
*   “在学习/写作业。”
*   “在做家务（比如做饭、打扫）。”
*   “在开会。”
*   “在路上（比如开车、坐地铁）。”

**3. 比较随意的朋友式回答：**
*   “摸鱼呢。”（表示在偷懒）
*   “在想你呀。”（开玩笑或调情）
*   “在思考人生。”（略带幽默）
*   “忙着呢，怎么了？”（直接且带有关心）

**总结一下：**
这个问题通常只是一个随口的问候或开场白，回答时不需要非常精确，根据你和对方的关系以及你当下的状态，选择一个最简单、最自然的说法即可。最常见的回答就是 **“没做什么”** 或直接说出你手头正在做的事。


In [22]:
# 绑定运行时参数, Runnable.bind()

# prompt = ChatPromptTemplate.from_messages([
#     (
#         'system',
#         '用代数符号写出下面的等式，并解决它。使用格式\n\nEQUATION:...\nSOLUTION:...\n\n',
#     ),
#     (
#         'human',
#         '{equation_statement}'
#     )
# ])

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)

llm = ChatOpenAI(base_url='https://api.deepseek.com', api_key=os.getenv('DEEPSEEK_SK'), model='deepseek-chat', temperature=0)

runnable = (
    {'equation_statement': RunnablePassthrough()}
    | prompt
    | llm.bind(stop='SOLUTION')
    | StrOutputParser()
)

result = runnable.invoke('x raised to the third plus seven equals 12')
# result = runnable.invoke('x减3乘7等于12')
print(result)


Let's break this down step by step.

---

**Step 1: Translate words into an equation**  
"x raised to the third" means \( x^3 \)  
"plus seven" means \( + 7 \)  
"equals 12" means \( = 12 \)  

So the equation is:  
\[
x^3 + 7 = 12
\]

---

**Step 2: Solve for \( x \)**  
Subtract 7 from both sides:  
\[
x^3 = 12 - 7
\]
\[
x^3 = 5
\]

Take the cube root of both sides:  
\[
x = \sqrt[3]{5}
\]

---

**Final answer:**

EQUATION: \( x^3 + 7 = 12 \)  



In [23]:
# 根据输入动态路由逻辑
# RunnableBranch使用一对（条件，可运行）和一个默认可运行的列表进行初始化。它通过将每个条件传递给其调用的输入，来选择哪个分支。它选择第一个计算结果为True的条件，并使用输入运行相应的可运行

from langchain_core.prompts import PromptTemplate
# from langchain_community.chat_models import ChatAnthropic

# 创建一个链，将传入的问题标识为LangChain、Anthropic、Other
prompt = PromptTemplate.from_template(
    '''针对以下给出的问题，区分他们，并回答：LangChain，Anthropic，Other。
    不要回复超过一个单词。
    <question>
    {question}
    </question>
    '''
)

chain = prompt | llm | StrOutputParser()
# result = chain.invoke({'question': '我应该怎么使用Anthropic'})
# print(result)

Anthropic


In [26]:
# 创建三个子链

langchain_chain = (
    PromptTemplate.from_template(
        '''
            你是一名使用LangChain的专家。请以中文回答问题。
            总是以"As Harrison Chase told me"作为回答的开头。
            回答以下的问题:
            Question: {question},
            Answer:
        '''
    ) | llm | StrOutputParser()
)


anthropic_chain = (
    PromptTemplate.from_template(
        '''
            你是一名使用Anthropic的专家。请以中文回答问题。
            总是以"As Dario Amodei told me"作为回答的开头。
            回答以下的问题:
            Question: {question},
            Answer:
        '''
    ) | llm | StrOutputParser()
)


general_chain = (
    PromptTemplate.from_template(
        '''
            以中文回答以下的问题:
            Question: {question},
            Answer:
        '''
    ) | llm | StrOutputParser()
)

from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: 'anthropic' in x['topic'].lower(), anthropic_chain),
    (lambda x: 'langchain' in x['topic'].lower(), langchain_chain),
    general_chain
)

full_chain = {'topic': chain, 'question': lambda x: x['question']} | branch

result = full_chain.invoke({'question': '我该怎么使用Anthropic'})
print(result)
print('\n\n')

result = full_chain.invoke({'question': '我该怎么使用Java'})
print(result)
print('\n\n')

result = full_chain.invoke({'question': '我该怎么使用Langchain'})
print(result)
print('\n\n')


As Dario Amodei told me, 使用Anthropic的方法主要取决于您想要使用的具体产品或服务。对于Anthropic的AI助手Claude，您可以通过以下方式使用：

1. 访问Claude官网或下载移动应用程序，直接与AI对话
2. 通过API接口将Claude集成到您的应用程序中
3. 在Slack工作区中使用Claude作为协作助手

对于开发者，建议查阅Anthropic官方文档了解详细的API使用指南和最佳实践。无论您选择哪种方式，都要确保遵循Anthropic的使用政策和安全准则，以负责任的方式利用AI技术。



在 Java 中，`super` 和 `this` 是两个特殊的关键字，用于在类的继承和对象构造中引用其他相关部分。以下是它们的主要区别：

## 1. `this` 关键字

`this` 代表当前对象的引用，主要用于：

### 引用当前对象的成员变量
```java
public class Person {
    private String name;
    
    public Person(String name) {
        this.name = name; // 区分参数和成员变量
    }
}
```

### 调用当前类的其他构造方法
```java
public class Person {
    private String name;
    private int age;
    
    public Person() {
        this("未知", 0); // 调用另一个构造方法
    }
    
    public Person(String name, int age) {
        this.name = name;
        this.age = age;
    }
}
```

### 作为方法参数传递当前对象
```java
public void display() {
    someMethod(this); // 传递当前对象
}
```

## 2. `super` 关键字

`super` 代表父类对象的引用，主要用于：

### 调用父类的构造方法
```java
public class Student e